In [377]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy import stats
import seaborn as sns
from datetime import datetime
import random
import math
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
# Reduce decimal points to 2
pd.options.display.float_format = '{:,.2f}'.format

import statistics 
from statistics import mode 
import fuzzywuzzy
from fuzzywuzzy import process
from collections import Counter

import os
#import psycopg2
import getopt
import sys
import math
import decimal
import copy
import csv
import progressbar
import time
#from toolkit import Date
from datetime import timedelta
import networkx as nx
#from network_viz import toGraph

import json 
import itertools

#from network_viz import output_Network_Viz
#from network_viz import get_Network_Viz_Monthly

In [378]:
start_date = datetime.strptime('2020-01-25', '%Y-%m-%d')
#end_date = datetime.strptime('2020-04-04', '%Y-%m-%d')
end_date = datetime.strptime(datetime.today().strftime("%Y-%m-%d"), "%Y-%m-%d")

### Read userData csv and save as dict

In [379]:
userData_raw = pd.read_csv('/Users/rebeccamqamelo/Desktop/Capstone/users_all_pub_Nov2020.csv', header=0, index_col=None)

In [380]:
# Apply correct datetime format
userData_raw['start'] = pd.to_datetime(userData_raw['start'], format='%Y-%m-%d').dt.floor('d')
userData_raw.sort_values(by='start', inplace=True)

# Fix messy genders
nans = userData_raw.index[userData_raw['gender'].isnull()].tolist()
userData_raw.loc[(userData_raw.index.isin(nans)) & (userData_raw.held_roles != 'ADMIN'), 'gender'] = 'unknown gender'
userData_raw.loc[(userData_raw.index.isin(nans)) & (userData_raw.held_roles == 'ADMIN'), 'gender'] = 'system'
userData_raw.gender = userData_raw.gender.str.lower()
userData_raw.gender = userData_raw.gender.replace('male ', 'male')

userData_raw.to_csv('/Users/rebeccamqamelo/Desktop/Capstone/userData_raw_sorted.csv')

In [400]:
reader = csv.DictReader(open('/Users/rebeccamqamelo/Desktop/Capstone/userData_raw_sorted.csv'))

userData_dict = {}
for row in reader:
    key = row.pop('xDAI_blockchain_address')
    if key in userData_dict:
        # implement your duplicate row handling here
        pass
    userData_dict[key] = row

### Read txnData csv and save as dict

In [382]:
txnData_raw = pd.read_csv('/Users/rebeccamqamelo/Desktop/Capstone/tx_all_pub_Nov2020.csv', header=0, index_col=None)

In [383]:
# Apply correct datetime format
txnData_raw['timeset'] = pd.to_datetime(txnData_raw['timeset'], format='%Y-%m-%d').dt.floor('d')
txnData_raw.sort_values(by='timeset', inplace=True)

# Fix messy genders
s_nans = txnData_raw.index[txnData_raw['s_gender'].isnull()].tolist()
txnData_raw.loc[(txnData_raw.index.isin(nans)), 's_gender'] = 'unknown gender'
txnData_raw.loc[(txnData_raw.s_business_type == 'System'), 's_gender'] = 'system'
txnData_raw.s_gender = txnData_raw.s_gender.str.lower()
txnData_raw.s_gender = txnData_raw.s_gender.replace('male ', 'male')

t_nans = txnData_raw.index[txnData_raw['t_gender'].isnull()].tolist()
txnData_raw.loc[(txnData_raw.index.isin(nans)), 't_gender'] = 'unknown gender'
txnData_raw.loc[(txnData_raw.t_business_type == 'System'), 't_gender'] = 'system'
txnData_raw.t_gender = txnData_raw.t_gender.str.lower()
txnData_raw.t_gender = txnData_raw.t_gender.replace('male ', 'male')

txnData_raw.to_csv('/Users/rebeccamqamelo/Desktop/Capstone/txnData_raw_sorted.csv')

In [401]:
# This cell takes approx. 8 min to run!
reader = csv.DictReader(open('/Users/rebeccamqamelo/Desktop/Capstone/txnData_raw_sorted.csv'))

txnData_dict = {}
for row in reader:
    date = row['timeset']
    sender_key = row['source']
    receiver_key = row['target']
    if sender_key in txnData_dict: # user's dict has already been created
        txnData_dict[sender_key].append({date: row})
    else:
        txnData_dict[sender_key] = [] # empty array to contain dict of each txn for that user
        txnData_dict[sender_key].append({date: row}) 
        
    if receiver_key in txnData_dict: # other user's dict has already been created
        txnData_dict[receiver_key].append({date: row})
    else:
        txnData_dict[receiver_key] = [] # empty array to contain dict of each txn for the other user
        txnData_dict[receiver_key].append({date: row}) 

### Update userData by time step

In [402]:
userData = userData_dict
txnData = txnData_dict
#userData = dict(itertools.islice(userData_dict.items(), 30))  
#txnData = dict(itertools.islice(txnData_dict.items(), 30))  

headersUserPub = ['id', 'start', 'label', 'gender', 'location', 'lat','lon', 
              'held_roles', 'business_type', 'bal', 'xDAI_blockchain_address', 
              'ovol_in', 'ovol_out', 'otxns_in', 'otxns_out', 'ounique_in', 'ounique_out', 
              'svol_in', 'svol_out', 'stxns_in', 'stxns_out', 'sunique_in', 'sunique_out',
              'sunique_out_group', 'sunique_in_at', 'sunique_out_at', 'sunique_out_at_group',
              'days_enrolled', 'days_active', 'trades_in_daily', 'trades_out_daily', 
              'trade_partners_in_daily', 'trade_partners_out_daily', 'gender_ratio_in', 'gender_ratio_out']

In [405]:
def create_userData(userData, txnData, start_date, end_date, headersUserPub):

    for user in userData.keys():
        volume_in = 0
        volume_out = 0
        txns_in = 0 # all trades
        txns_out = 0
        unique_txns_in = 0 # unique trades
        unique_txns_out = 0

        svolume_in = 0
        svolume_out = 0
        stxns_out = 0
        stxns_in = 0
        sunique_txns_out = 0
        sunique_txns_out_group = 0
        sunique_txns_in = 0

        sunique_txns_out_atleast = 0
        stotal_unique_txns_out_atleast = 0
        stotal_unique_txns_out_atleast_group = 0
        #stotal_unique_txns_out_atleast_group += 1
        sunique_txns_out_atleast_group = 0
        sunique_txns_in_atleast = 0
        
        females_sold_to = 0
        males_sold_to = 0
        females_bought_from = 0
        males_bought_from = 0

        min_size = 5 # min Sarafu trade amount (?)

        sseenRecUsers = []
        seenRecUsers = []

        sseenSentUsers = []
        seenSentUsers = []

        if(user in txnData.keys()): # if user has participated in a trade in the txnData dict
            txns = txnData[user] # list containing dicts for each txn
            #for trans in txnData[user]: # for each txn in the user's txn array
            for txn in txns: # parses each txn dict, where the key is the date
                date_str = list(txn.keys())[0]
                date = datetime.strptime(date_str, '%Y-%m-%d')
                if start_date <= date <= end_date: # checks that the date is within the defined range
                    trans = txn[date_str]
                    #for trans in txn:
                    if trans['source'] == user:
                        if trans['transfer_subtype'] != 'STANDARD':
                            volume_out+=float(trans['weight'])
                            txns_out+=1
                            if trans['target'] not in seenRecUsers:
                                seenRecUsers.append(trans['target'])
                                unique_txns_out+=1

                        else: # standard txn with other users
                            svolume_out+=float(trans['weight'])
                            stxns_out+=1
                            
                            if userData[trans['target']]['gender'] == 'female':
                                females_sold_to+=1
                            elif userData[trans['target']]['gender'] == 'male':
                                males_sold_to+=1
                            
                            #if (user == 4082):
                            #    print(stxns_out)
                            if trans['target'] not in sseenRecUsers: # txn is standard and unqiue
                                sseenRecUsers.append(trans['target'])
                                sunique_txns_out+=1
                                if userData[user]['held_roles'] == "GROUP_ACCOUNT":
                                    sunique_txns_out_group += 1
                                if(float(trans['weight'])>=min_size):
                                    sunique_txns_out_atleast += 1
                                    if sunique_txns_out_atleast > 1:
                                        stotal_unique_txns_out_atleast += 1
                                    if userData[user]['held_roles'] == "GROUP_ACCOUNT":
                                        sunique_txns_out_atleast_group += 1
                                        if sunique_txns_out_atleast_group > 1:
                                            stotal_unique_txns_out_atleast_group += 1


                    else: # this txn marks where the user was a recipient
                        if trans['transfer_subtype'] != 'STANDARD':
                            #if user == 13488:
                            #    print("<><><><> ",trans['_transfer_amount_wei'], trans['created'])
                            volume_in+=float(trans['weight'])
                            txns_in+=1
                            
                            if userData[trans['source']]['gender'] == 'female':
                                females_bought_from+=1
                            elif userData[trans['source']]['gender'] == 'male':
                                males_bought_from+=1
                            
                            if trans['source'] not in seenSentUsers:
                                seenSentUsers.append(trans['source'])
                                unique_txns_in+=1
                            
                        else:
                            svolume_in+=float(trans['weight'])
                            stxns_in+=1
                            if trans['source'] not in sseenSentUsers:
                                sseenSentUsers.append(trans['source'])
                                sunique_txns_in+=1
                                if(float(trans['weight'])>=min_size):
                                    sunique_txns_in_atleast+=1

                                    
            # kinda bad practice: using individual txn keys, which are dates, to parse first and last txn
            # NB first txn is NOT 0-indexed because this would be their disbursement i.e registration txn
            # – this doesn't count as "active trading"
            
            if len(txnData[user]) == 1: 
                first_trade_date = datetime.strptime(list(txnData[user][0].keys())[0], '%Y-%m-%d') 
            elif len(txnData[user]) > 1:
                first_trade_date = datetime.strptime(list(txnData[user][1].keys())[0], '%Y-%m-%d') 
            last_trade_date = datetime.strptime(list(txnData[user][-1].keys())[0], '%Y-%m-%d')
            #start = datetime.strptime(userData[user]['start'].to_string(index=False), '%Y-%m-%d')
            start = datetime.strptime(userData[user]['start'], '%Y-%m-%d')
            end = end_date

            days_enrolled = (end - start).days
            days_active = abs(last_trade_date - first_trade_date).days
            
            # Avoid division by zero:
            if days_enrolled == 0:
                days_enrolled = 1
            if days_active == 0:
                days_active = 1

            trades_in_daily = stxns_in/days_active
            trades_out_daily = stxns_out/days_active
            trade_partners_in_daily = sunique_txns_in/days_active
            trade_partners_out_daily = sunique_txns_out/days_active
            
            # Avoid division by zero:
            if males_sold_to == 0:
                males_sold_to = 1
            if males_bought_from == 0:
                males_bought_from = 1
                
            gender_ratio_out = females_sold_to/males_sold_to
            gender_ratio_in = females_bought_from/males_bought_from
            #total_unique_out_atleast += stotal_unique_txns_out_atleast
        
        txData = {'ovol_in':volume_in, 'ovol_out':volume_out, 'otxns_in':txns_in, 'otxns_out':txns_out,
                  'ounique_in':unique_txns_in, 'ounique_out':unique_txns_out, 'svol_in':svolume_in, 
                  'svol_out':svolume_out, 'stxns_in':stxns_in, 'stxns_out':stxns_out, 
                  'sunique_in':sunique_txns_in, 'sunique_out':sunique_txns_out,
                  'sunique_out_group':sunique_txns_out_group, 
                  'sunique_in_at':sunique_txns_in_atleast,
                  'sunique_out_at':sunique_txns_out_atleast, 
                  'sunique_out_at_group':sunique_txns_out_atleast_group,
                  'days_enrolled': days_enrolled, 
                  'days_active': days_active, 
                  'trades_in_daily': trades_in_daily, 
                  'trades_out_daily': trades_out_daily, 
                  'trade_partners_in_daily': trade_partners_in_daily, 
                  'trade_partners_out_daily': trade_partners_out_daily,
                  'gender_ratio_in': gender_ratio_in,
                  'gender_ratio_out': gender_ratio_out}

        uDict = userData[user]
        uDict.update(txData)
        userData[user] = uDict

In [406]:
# This cell takes approx. _ min t run!
create_userData(userData, txnData, start_date, end_date, headersUserPub)

In [407]:
def generate_user_and_transaction_data_github_csv(txnData, userData, unique_txnData, start_date, end_date, days_ago_str, headersUserPub, private=False):

    headersTxPub = ['id', 'timeset', 'transfer_subtype', 'transfer_use','source', 's_comm_tkn', 's_gender', 's_location_path', 's_location_lat','s_location_lon',
                     's_business_type', 'target', 't_comm_tkn', 't_gender',
                    't_location_path', 't_location_lat', 't_location_lon',
                     't_business_type', 'tx_token', 'weight', 'type','token_name', 'token_address']

    headersUser = headersUserPub
    headersTx = headersTxPub
    
    #filenameTx = '/Users/rebeccamqamelo/Desktop/Capstone/tx_all_pub_'+start_date.strftime("%Y-%m-%d")+"-"+end_date.strftime("%Y-%m-%d")+"-"+days_ago_str+'.csv'
    #filenameUser = '/Users/rebeccamqamelo/Desktop/Capstone/users_all_pub_'+start_date.strftime("%Y-%m-%d")+"-"+end_date.strftime("%Y-%m-%d")+"-"+days_ago_str+'.csv'
    filenameTx = '/Users/rebeccamqamelo/Desktop/Capstone/tx_all_pub_20200125-20201109-all_time.csv'
    filenameUser = '/Users/rebeccamqamelo/Desktop/Capstone/users_all_pub_20200125-20201109-all_time.csv'
    
    print("saving all transactions to: ", filenameTx)
    print("saving all users to: ", filenameUser)

    #timestr = time.strftime("%Y%m%d-%H%M%S")
    timestr = time.strftime("%Y-%m-%d")

    token_transactions = txnData

    indexR = 0

    seen_users = []
    exclude_list = []

    if True:
        numberTx = 1
        numberUsers = 0

        with open(filenameTx, 'w', newline='') as csvfileTx, open(filenameUser, 'w', newline='') as csvfileUser:
            #spamwriterTx = csv.writer(csvfileTx)
            #spamwriterTx.writerow(headersTx)

            spamwriterUser = csv.writer(csvfileUser)
            spamwriterUser.writerow(headersUser)

            for user_id, user_info in userData.items():

                user_data1 = {'start': user_info['start']}
                user_data1['id'] = user_info['id']
                user_data1['label'] = user_info['id']
                user_data1['xDAI_blockchain_address'] = user_id
                #user_data1['xDAI_blockchain_address'] = user_info['xDAI_blockchain_address']

                #user_data1['comm_tkn'] = user_info['comm_tkn']

                user_data1['bal'] = user_info['bal']
                user_data1['gender'] = user_info.get('gender')
                user_data1['location'] = user_info.get('location')
                user_data1['lat'] = user_info.get('lat')
                user_data1['lon'] = user_info.get('lon')
                user_data1['held_roles'] = user_info['held_roles']
                user_data1['business_type'] = user_info['business_type']
                user_data1['ovol_in'] = user_info['ovol_in']
                user_data1['ovol_out'] = user_info['ovol_out']
                user_data1['otxns_in'] = user_info['otxns_in']
                user_data1['otxns_out'] = user_info['otxns_out']
                user_data1['ounique_in'] = user_info['ounique_in']
                user_data1['ounique_out'] = user_info['ounique_out']
                user_data1['svol_in'] = user_info['svol_in']
                user_data1['svol_out'] = user_info['svol_out']
                user_data1['stxns_in'] = user_info['stxns_in']
                user_data1['stxns_out'] = user_info['stxns_out']
                user_data1['sunique_in'] = user_info['sunique_in']
                user_data1['sunique_out'] = user_info['sunique_out']
                user_data1['sunique_out_group'] = user_info['sunique_out_group']
                user_data1['sunique_in_at'] = user_info['sunique_in_at']
                user_data1['sunique_out_at'] = user_info['sunique_out_at']
                user_data1['sunique_out_at_group'] = user_info['sunique_out_at_group']
                user_data1['days_enrolled'] = user_info['days_enrolled']
                user_data1['days_active'] = user_info['days_active']
                user_data1['trades_in_daily'] = user_info['trades_in_daily']
                user_data1['trades_out_daily'] = user_info['trades_out_daily']
                user_data1['trade_partners_in_daily'] = user_info['trade_partners_in_daily']
                user_data1['trade_partners_out_daily'] = user_info['trade_partners_out_daily']
                user_data1['gender_ratio_in'] = user_info['gender_ratio_in']
                user_data1['gender_ratio_out'] = user_info['gender_ratio_out']
                
                #user_data1['confidence'] = user_info['confidence']

                numberUsers+=1

                spamwriterUser.writerow([str(user_data1[k]) for k in headersUser])

            c_idx = 0
            chunks = 10000
            tx_hash = []

            """
            #for tnsfer_acct__id, transactions in txnData.items():
            if len(unique_txnData) > 0:
                for key, t in unique_txnData.items():
                #for t in unique_txnData:
                    #if t['id'] in tx_hash:  # only looking at unique data: note that without this there will be double counting on transactions
                    #    print("Error: duplicate transaction found!", t['id'])
                    #    time.sleep(1.5)
                    #    continue
                    #tx_hash.append(t['id'])

                    sender_user_id = t['source']
                    print(sender_user_id)
                    recipient_user_id = t['target']
                    print(recipient_user_id)

                    if sender_user_id == None:
                        sender_user_id = 1
                    #elif sender_user_id == '0xEDA5C9B75Fdb3B9bdAB987A704632280Cf93084F': # Grassroots Economics System
                        #sender_user_id = 1

                    if recipient_user_id == None:
                        recipient_user_id = 1

                    row_data = {'timeset': t['timeset']}

                    row_data['weight'] = t['weight']
                    row_data['type'] = 'directed'
                    row_data['transfer_subtype'] = t['transfer_subtype']
                    row_data['id'] = t['id']
                    #row_data['label'] = t['label']
                    #if private:
                        #row_data['tx_hash'] = t['tx_hash']
                    row_data['token_name'] = t['token_name']
                    row_data['authorising_user_id'] = t.get('authorising_user_id') #t['authorising_user_id']
                    row_data['token_address'] = t['token_address']
                    
                    #if t['transfer_use'] != None:
                        #row_data['transfer_use'] = t['transfer_use'][0].strip('"[]')
                    #else:
                        #row_data['transfer_use'] = ''
                    row_data['transfer_use'] = t['transfer_use']

                    #row_data['source'] = userData[sender_user_id]['blockchain_address']
                    row_data['source'] = userData[sender_user_id]['xDAI_blockchain_address']
                    row_data['s_location_path'] = userData[sender_user_id].get('s_location_path')
                    row_data['s_location_lat'] = userData[sender_user_id].get('s_location_lat')
                    row_data['s_location_lon'] = userData[sender_user_id].get('s_location_lon')
                    row_data['t_location_path'] = userData[recipient_user_id].get('t_location_path')
                    row_data['t_location_lat'] = userData[recipient_user_id].get('t_location_lat')
                    row_data['t_location_lon'] = userData[recipient_user_id].get('t_location_lon')
                    row_data['s_gender'] = userData[sender_user_id].get('s_gender')
                    row_data['s_comm_tkn'] = userData[sender_user_id]['s_comm_tkn']
                    row_data['s_business_type'] = userData[sender_user_id]['s_business_type']

                    row_data['target'] = userData[recipient_user_id]['target']
                    if True:
                        row_data['t_gender'] = userData[recipient_user_id].get('gender')
                        row_data['t_comm_tkn'] = userData[recipient_user_id]['t_comm_tkn']
                        row_data['t_business_type'] = userData[recipient_user_id]['t_business_type']

                    if True: #Admin ONLY row_data['authorising_user_id'] is not None and row_data['authorising_user_id'] != 6: #Admin ONLY
                        rowString = []
                        for k in headersTx:
                            if k in row_data.keys():
                                #spamwriterTx.writerow([str(row_data[k]) for k in headersTx])
                                rowString.append(str(row_data[k]))
                            else:
                                rowString.append('')

                        spamwriterTx.writerow(rowString)
                        if indexR < 3:
                            #print(row_data) #debug
                            indexR+=1
                        if c_idx >= chunks:
                            c_idx = 0
                            csvfileTx.flush()  # whenever you want
                            print("chunk: ", numberTx)
                        else:
                            c_idx += 1

                        numberTx += 1

        print("****saved all transactions to csv", filenameTx, " number of tx:", numberTx, timestr)
            
            """  
            
        print("****saved all users to csv", filenameUser, " number of User:", numberUsers, timestr)

In [408]:
unique_txnData = txnData.copy()
days_ago_str = "all_time"
generate_user_and_transaction_data_github_csv(txnData, userData, unique_txnData, start_date, end_date, days_ago_str, headersUserPub, private=False)

saving all transactions to:  /Users/rebeccamqamelo/Desktop/Capstone/tx_all_pub_20200125-20201109-all_time.csv
saving all users to:  /Users/rebeccamqamelo/Desktop/Capstone/users_all_pub_20200125-20201109-all_time.csv
****saved all users to csv /Users/rebeccamqamelo/Desktop/Capstone/users_all_pub_20200125-20201109-all_time.csv  number of User: 40941 2020-11-13
